In [1]:
import os
import sys
from collections import OrderedDict

In [2]:
sys.path.append("../..")

In [3]:
%load_ext autoreload
%autoreload 2

In [7]:
import spatialpy

In [8]:
import matplotlib.pyplot as plt
import numpy

# Global Constants
MAX_X_DIM = 5.0
MIN_X_DIM = -5.0
TOL = 1e-9

In [9]:
class Edge1(spatialpy.SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0] - MAX_X_DIM) < 0.05
class Edge2(spatialpy.SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0] - MIN_X_DIM) < 0.05


In [10]:
!ls

cylinder.xml  old  SpatialPy_cylinderDemo3D.ipynb


In [11]:
numpy.zeros((0))

array([], dtype=float64)

In [21]:
class cylinderDemo3D(spatialpy.Model):
    def __init__(self, model_name="cylinder_demo3d"):
        spatialpy.Model.__init__(self, model_name)

        # System constants
        D_const = 0.1

        # Define Species
        A = spatialpy.Species(name="A", diffusion_constant=D_const)
        B = spatialpy.Species(name="B", diffusion_constant=D_const)
        self.add_species([A, B])

        # Define Geometry
        self.mesh = spatialpy.Mesh.read_xml_mesh('cylinder.xml')

        # Define Subdomains
        self.add_subdomain(Edge1(), 2)
        self.add_subdomain(Edge2(), 3)
        
        # Restrict the movement of Chemical Species
        self.restrict(A,[1,2])
        self.restrict(B,[1,3])


        k_react = spatialpy.Parameter(name="k_react", expression=1.0)
        k_creat1 = spatialpy.Parameter(name="k_creat1", expression=100)
        k_creat2 = spatialpy.Parameter(name="k_creat2", expression=100)
        self.add_parameter([k_react, k_creat1,k_creat2])


        # Define Reactions
        R1 = spatialpy.Reaction(reactants=None, products={A:1}, 
                                rate=k_creat1, restrict_to=2)
        R2 = spatialpy.Reaction(reactants=None, products={B:1}, 
                              rate=k_creat2, restrict_to=3)
        R3 = spatialpy.Reaction(reactants={A:1, B:1}, products=None, 
                              rate=k_react)
        self.add_reaction([R1, R2, R3])

        # Define simulation timespan
        self.set_timesteps(1, 200)

In [22]:
model = cylinderDemo3D()

In [26]:
result = model.run(report_level=2)

Compiling Solver
Creating propensity file /tmp/tmpwhqdgkpd/.spatialpy/cylinder_demo3d_generated_model.c
cmd: cd /tmp/tmpwhqdgkpd ; make -f /home/brian/Desktop/research/SpatialPy/spatialpy/spatialpy/build/Makefile.nsm SpatialPy_ROOT=/home/brian/Desktop/research/SpatialPy/spatialpy/spatialpy SpatialPy_MODEL=cylinder_demo3d_generated_model

b''
b"make: /home/brian/Desktop/research/SpatialPy/spatialpy/spatialpy/build/Makefile.nsm: No such file or directory\nmake: *** No rule to make target '/home/brian/Desktop/research/SpatialPy/spatialpy/spatialpy/build/Makefile.nsm'.  Stop.\n"


SimulationError: Compilation of solver failed, return_code=2